# Parameters selection

- Model category
- Features
- Layers
- Cells
- Regularization
- Batch size
- Timesteps

### Neural Network Layers and Neuron Counts

| Layer 1 (Input Layer)     | Layer 2 (Hidden Layer) | Layer 3 (Hidden Layer) |
|---------------------------|------------------------|------------------------|
| 5                         | 0                      | 0                      |
| 10                        | 10                     | 10                     |
| 20                        | 20                     | 15                     |
| 30                        | 50                     | 20                     |
| 50                        |                        |                        |
| 75                        |                        |                        |
| 100                       |                        |                        |
| 125                       |                        |                        |
| 150                       |                        |                        |


In [ ]:

# Model category name used throughout the subsequent analysis
model_catg_id = "01"

# Load features from dataset:
# ['all', 'actual', 'entsoe', 'weather_t', 'weather_i', 'holiday', 'weekday', 'hour', 'month']
features = ['actual', 'entsoe']

# LSTM Layer configuration
# ========================
# Stateful True or false
layer_conf = [True, True, True]
# Number of neurons per layer
cells = [[ 5, 10, 20, 30, 50, 75, 100, 125, 150 ], [0, 10, 20, 50], [0, 10, 15, 20]]
# Regularization per layer
dropout = [0, 0.1, 0.2]
# Size of how many samples are used for one forward/backward pass
batch_size = [8]
# In a sense this is the output neuron dimension, or how many timesteps the neuron should output. Currently not implemented, defaults to 1.
timesteps = [1]

Packages update its usage:  [statsmodels](https://www.statsmodels.org/stable/tsa.html)

In [ ]:
import os
import sys
import math
import itertools
import datetime as dt
import pytz
import time as t
import numpy as np
import pandas as pd
from pandas import read_csv
# from pandas import datetime
from numpy import newaxis

import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
# from statsmodels.tsa import stattools
import statsmodels.api as stattools
from tabulate import tabulate
import math

import keras as keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
from keras.callbacks import TensorBoard
from keras.utils import np_utils
from keras.models import load_model

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from IPython.display import HTML
from IPython.display import display


# %matplotlib - max figure size
mpl.rcParams['figure.figsize'] = (9.0,5.0)

# Import custom module functions
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lstm_load_forecasting import data, lstm

## Overall Configuration

In [ ]:
path = os.path.join(os.path.abspath(''), '../data/fulldataset.csv')

# Split date for train and test data. 
# As the TBATS and ARIMA benchmark needs 2 full cycle of all seasonality, needs to be after jan 01. 
loc_tz = pytz.timezone('Europe/Zurich')
split_date = loc_tz.localize(dt.datetime(2017,2,1,0,0,0,0))

# Validation split percentage
validation_split = 0.2
# How many epochs in total
epochs = 30
# Set verbosity level. 0 for only per model, 1 for progress bar...
verbose = 0

# Dataframe containing the relevant data from training of all models
results = pd.DataFrame(columns=['model_name', 'config', 'dropout',
                                'train_loss', 'train_rmse', 'train_mae', 'train_mape', 
                                'valid_loss', 'valid_rmse', 'valid_mae', 'valid_mape', 
                                'test_rmse', 'test_mae', 'test_mape',
                                'epochs', 'batch_train', 'input_shape',
                                'total_time', 'time_step', 'splits'
                               ])
# Early stopping parameters
early_stopping = True
min_delta = 0.006
patience = 2

## Model Generation

In [ ]:
# Output path
res_dir = '../results/notebook_' + model_catg_id + '/'
plot_dir = '../plots/notebook_' + model_catg_id + '/'
model_dir = '../models/notebook_' + model_catg_id + '/'
os.makedirs(res_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
